In [25]:
!pip install numpy pandas matplotlib seaborn scikit-learn nltk 

Defaulting to user installation because normal site-packages is not writeable


In [27]:

import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

This notebook is for **fraud call detection** dataset. the dataset contain two type of data 
one is fraud and second is normal call data.
To detect fraud call, In this notebook i am going to used Naive Bayes classification.

when ever we start creating a model in Machine Learning the first step is perform EDA ,in the EDA first step is import/load the dataset.

In [28]:
# """import the dataset in data library"""
data = pd.read_csv("fraud_call.file",sep='\t',on_bad_lines='skip',names=['label','content'])

In [29]:
data.head()

,label,content
0,fraud,"hello, i m bank manager of SBI, ur debit card ..."
1,fraud,Todays Vodafone numbers ending with 4882 are s...
2,normal,Please don't say like that. Hi hi hi
3,normal,Thank you!
4,normal,Oh that was a forwarded message. I thought you...


I am going to count howmany fraud and normal data is avialble in dataset.

In [30]:
data['label'].value_counts()

label
normal    5515
fraud      875
Name: count, dtype: int64

visualize the label column **for count** the number of fraud data and normal data.

In [31]:
import seaborn as sns

In **machine learning technique**, Every model will take input as integer/numeric data type values,but in this dataset data are present in object type so before giving input to model we have to convert object type data to integer types.

In the below cell i have used **TfidVectorizer method** which is imported from nltk library for create a vector of words which is present dataset. **TfidVectorizer** is more better then **Bag of Words(BOW)** method.

***Note:*** for categorical/object type information, before performing encoding methods always remove stopwords from the data because  stopwords do not play important role for improve performance and accuracy of model.

so in the below cell, for removing stopwords "WordNetLemmatizer" have used. it will remove stopwords from dataset.

Ex: for,in,you,at etc these are stopwords,
**stopwords** imported from **nltk** library, this is a text Preprocessing library. 



In [32]:
"""perform text preprocessing"""
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
ps = WordNetLemmatizer()
cv = TfidfVectorizer(max_features=2000)

In [33]:
def remove_digit(data) :
    corpos = []
    for i in range(0, len(data)) :
        review = re.sub('[^a-zA-Z]', ' ', data['content'][i])
        review = review.lower()
        review = review.split()
        review = [ps.lemmatize(word) for word in review if word not in stopwords.words('english')]
        review = ' '.join(review)
        corpos.append(review)
    return corpos


import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ycham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ycham\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
import pandas as pd
from sklearn.metrics import recall_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

# Assuming 'cv' is your CountVectorizer instance
cv = CountVectorizer()

def detect_model(corpos, data):
    x = cv.fit_transform(corpos).toarray()
    y = pd.get_dummies(data['label'])
    y = y.iloc[:, 1].values
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)
    fraud_detect = MultinomialNB().fit(x_train, y_train)
    print("Model has trained.")
    y_pred = fraud_detect.predict(x_test)
    cong_m = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    cl_r = classification_report(y_test, y_pred)
    
    print("Confusion matrix:", cong_m)
    print("Accuracy score:", acc)
    print("Recall score:", recall)
    print("Classification report:\n", cl_r)
    return fraud_detect

proper_list = remove_digit(data)
model= detect_model(proper_list, data)


Model has trained.
Confusion matrix: [[ 146   10]
 [  36 1086]]
Accuracy score: 0.9640062597809077
Recall score: 0.9679144385026738
Classification report:
               precision    recall  f1-score   support

       False       0.80      0.94      0.86       156
        True       0.99      0.97      0.98      1122

    accuracy                           0.96      1278
   macro avg       0.90      0.95      0.92      1278
weighted avg       0.97      0.96      0.97      1278



In [35]:
import joblib

# Save the model
joblib.dump(model, 'model/model2.pkl')

# Save the vectorizer
joblib.dump(cv, 'model/vectorizer2.pkl')

print("Model and vectorizer have been saved.")

Model and vectorizer have been saved.


In [24]:
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Convert scikit-learn model to ONNX
initial_type = [('input', FloatTensorType([None, 4]))]  # Change the size based on your features
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save ONNX model
with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

from onnx_tf.backend import prepare

# Load ONNX model
onnx_model = onnx.load("model.onnx")

# Convert to TensorFlow
tf_rep = prepare(onnx_model)
tf_rep.export_graph("model_tf")

# Now convert to TensorFlow.js
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model model_tf/ model_tfjs/


ModuleNotFoundError: No module named 'onnx'

In  given below cell, **Naive Bayes** classification has used to classify whether it is fraud call or not.
before it split data in to two parts.

**1. training data
2. test data**

An in-build **train_test_split** module used to split data in to training and testing part.

In [ ]:
def predict_input(model, input_text):
    # Preprocess the input string (transform into feature vector)
    input_transformed = cv.transform([input_text]).toarray()  # Ensure the input is in list format
    
    # Get the predicted probabilities
    probabilities = model.predict_proba(input_transformed)

    # Assuming binary classification, probabilities[:, 1] gives the probability of the positive class
    positive_prob = probabilities[0, 1] if probabilities.shape[1] > 1 else 0.5  # Fallback to 0.5 for single class
    
    # Scale it to a range of 0 to 100
    score = round(positive_prob * 100)
    
    return 100 - score

# Example usage:
result = predict_input(model, "hello bank manager sbi ur debit card expire would u want issue new card")
print("Predicted probability of being fraud:", result)

Predicted probability of being fraud: 100


In [21]:
res = predict_input(model, "zero")
res

25

call **remove_digit()** for text preprocessing operation and **detect_model()** for creating classification model 
and train it by traing data as well as validate it by testing data and will return **confusion_metrics** and 
**accuracy_score**"""

In [10]:
"""when dataset is imbalanced dataset, then we should use precision,recall or f1-score metrics function because 
accuracy will be failed in unbalance dataset  for  classification. but some time it will work fine"""


'when dataset is imbalanced dataset, then we should use precision,recall or f1-score metrics function because \naccuracy will be failed in unbalance dataset  for  classification. but some time it will work fine'

## """if you all find out this notebook  helpful, please upvote"""

In [81]:
print(proper_list)

['hello bank manager sbi ur debit card expire would u want issue new card', 'today vodafone number ending selected receive award number match call receive award', 'please say like hi hi hi', 'thank', 'oh forwarded message thought send', 'got seventeen pound seven hundred ml hope ok', 'funny', 'sweetheart hope kind day one load reason smile biola', 'login date time dad fetching home', 'shower baby', 'askd u question hour answer', 'well definitely need restock thanksgiving let know', 'said kiss kiss sound effect gorgeous man kind person need smile brighten day', 'probably gonna swing wee bit', 'ya nice ready thursday', 'allo braved bus taken train triumphed mean b ham jolly good rest week', 'watching cartoon listening music amp eve go temple amp church', 'mind ask happened say uncomfortable', 'problem send email', 'cash prize claim call', 'thats cool sometimes slow gentle sometimes rough hard', 'gonna say sorry would normal starting panic time sorry seeing tuesday', 'hello sir jio custom

In [16]:
# !pip install sounddevice
# !pip install SpeechRecognition



import sounddevice as sd
import scipy.io.wavfile as wav
# import SpeechRecognition as sr
import numpy as np
import os
import wave
import json
from vosk import Model, KaldiRecognizer

# Path to the model folder
model_path = "vosk-model-small-en-us-0.15"

# Initialize the Vosk model
if not os.path.exists(model_path):
    print("Model not found!")
else:
    model = Model(model_path)

# Function to recognize speech from an audio file
def transcribe_audio(file_path):
    wf = wave.open(file_path, "rb")
    
    # Check if audio file has the correct format (16kHz, Mono)
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() not in [16000, 8000]:
        print("Audio file must be WAV format mono PCM.")
        return
    
    # Initialize recognizer
    rec = KaldiRecognizer(model, wf.getframerate())
    
    # Transcribe the audio
    results = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            results.append(result['text'])
    
    # Finalize and print the transcription
    final_result = json.loads(rec.FinalResult())
    results.append(final_result['text'])
    
    return " ".join(results)

# Example usage
transcription = transcribe_audio("recorded_audio.wav")
print("Transcribed Text:", transcription)


# Step 1: Record Audio
def record_audio(duration=15, fs=44100, filename="recorded_audio.wav"):
    print("Recording for", duration, "seconds...")
    # Record audio
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait for the recording to finish
    print("Recording finished.")
    
    # Save as WAV file
    wav.write(filename, fs, np.int16(audio * 32767))  # Save audio in WAV format
    print(f"Audio saved as {filename}")
    return filename

filename = record_audio(duration=5)  # Record for 5 seconds
# audio_to_text(filename)


ModuleNotFoundError: No module named 'vosk'